# Setup

In [1]:
import numpy as np
import string 
import re 
import spacy
import nltk
from nltk.corpus import stopwords 
from nltk import word_tokenize
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer 
import pandas as pd
from tqdm import tqdm

In [2]:
pd.set_option('display.max_colwidth',200)
tqdm.pandas()

nltk.download("stopwords")
nltk.download("punkt")
tqdm.pandas()

[nltk_data] Downloading package stopwords to /Users/aiman/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /Users/aiman/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [3]:
from src.load import load_data_as_df
from src.preprocessing import remove_stop_words
import config

# Load Data

In [5]:
en_file = "data/europarl-v7.nl-en.en"
nl_file = "data/europarl-v7.nl-en.nl"
%time
df = load_data_as_df(en_file, nl_file)

CPU times: user 1 μs, sys: 1e+03 ns, total: 2 μs
Wall time: 3.1 μs
Loading English Corpora from: data/europarl-v7.nl-en.en ...
Loading Dutch Corpora from: data/europarl-v7.nl-en.nl ...


In [6]:
len(df)

1997775

In [7]:
df.head()

,English,Dutch
0,Resumption of the session,Hervatting van de zitting
1,"I declare resumed the session of the European Parliament adjourned on Friday 17 December 1999, and I would like once again to wish you a happy new year in the hope that you enjoyed a pleasant fest...","Ik verklaar de zitting van het Europees Parlement, die op vrijdag 17 december werd onderbroken, te zijn hervat. Ik wens u allen een gelukkig nieuwjaar en hoop dat u een goede vakantie heeft gehad."
2,"Although, as you will have seen, the dreaded 'millennium bug' failed to materialise, still the people in a number of countries suffered a series of natural disasters that truly were dreadful.","Zoals u heeft kunnen constateren, is de grote ""millenniumbug"" uitgebleven. De burgers van een aantal van onze lidstaten zijn daarentegen door verschrikkelijke natuurrampen getroffen."
3,"You have requested a debate on this subject in the course of the next few days, during this part-session.",U heeft aangegeven dat u deze vergaderperiode een debat wilt over deze rampen.
4,"In the meantime, I should like to observe a minute' s silence, as a number of Members have requested, on behalf of all the victims concerned, particularly those of the terrible storms, in the vari...",Nu wil ik graag op verzoek van een aantal collega's een minuut stilte in acht nemen ter nagedachtenis van de slachtoffers. Ik doel hiermee met name op de slachtoffers van het noodweer dat verschil...


# Preprocessing

- lower case
- strip
- remove special characters
- remove digits
- remove duplicates
- remove rows with too short strings

In [9]:
def preprocess(text, stop_words):
    if not isinstance(text, str) or not text.strip():
        return ""
        
    # always lowercase & remove whitespaces
    text = text.lower().strip()

    # Remove everything after '<' if it exists
    if "<" in text:
        text = text.split("<")[0].strip()

    if not text:
        return ""  # Leave empty for later row drop

    # Remove punctuation
    if config.remove_punct:
        text = "".join(char for char in text if char not in string.punctuation)

    # Remove stopwords
    if config.remove_stopwords:
        tokens = word_tokenize(text)
        text = " ".join(word for word in tokens if word not in stop_words)

    # Remove numbers
    if config.remove_nums:
        text = re.sub(r"\d+", "", text)

    return text

In [33]:
def preprocess_dataframe(df):
    print(f"[Start] Total raw rows: {len(df)}")

    # Remove non-strings and nulls
    df = df[df["English"].apply(lambda x: isinstance(x, str))]
    df = df[df["Dutch"].apply(lambda x: isinstance(x, str))]
    df = df.dropna(subset=["English", "Dutch"])
    print(f"[Step 1] Rows after removing nulls/non-strings: {len(df)}")

    # Load stopwords
    en_stop = set(stopwords.words("english"))
    nl_stop = set(stopwords.words("dutch"))
    nl_stop.update(config.custom_nl_stopwords)

    # Preprocess text
    df["English"] = df["English"].progress_apply(lambda x: preprocess(x, en_stop))
    df["Dutch"] = df["Dutch"].progress_apply(lambda x: preprocess(x, nl_stop))


    # Drop duplicates
    df = df.drop_duplicates(keep="first")

    # Remove rows with too short strings
    df = df[(df["English"].str.strip() != "") & (df["Dutch"].str.strip() != "")]
    df = df[(df["English"].str.len() >= config.min_len_chars) & 
            (df["Dutch"].str.len() >= config.min_len_chars)]

    # Remove overly long sentences
    df = df[df.apply(
        lambda row: len(row["English"].split()) <= config.max_len_tokens and 
                    len(row["Dutch"].split()) <= config.max_len_tokens, axis=1)]

    print(f"[Step 2] Rows after cleaning/filtering: {len(df)}")

    return df

In [35]:
df_clean = preprocess_dataframe(df)

[Start] Total raw rows: 1997775
[Step 1] Rows after removing nulls/non-strings: 1997775


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1997775/1997775 [00:15<00:00, 129898.60it/s]


[Step 2] Rows after cleaning/filtering: 1930805


In [52]:
# Sample a fraction
df_sampled = df_clean.sample(frac=config.sample_frac, random_state=config.random_state).reset_index(drop=True)
df_sampled

,English,Dutch
0,these programmes have their purpose,de bestaande programma s hebben allemaal een doel en daarvoor hebben ze ook financiële ondersteuning gekregen niet voor iets anders
1,liberalisation is always going to be difficult,liberalisering verloopt immers altijd moeizaam
2,could you try to get the services to do something about the state of the lifts in this building,zou u kunnen proberen de diensten iets te laten doen aan de toestand van de liften in dit gebouw
3,is it true that after the failure of russia to make a real contribution to the talks with milosevic they will be willing to agree to military intervention if necessary this strikes me as a good th...,is het zo dat na het mislukken van een echte bijdrage van die kant in het gesprek met milosevic de bereidheid van de zijde van rusland groter zal zijn om in te stemmen met militaire actie indien n...
4,they are responsible for around of the jobs created,kmos verantwoordelijk zijn voor procent van alle gecreëerde banen
...,...,...
193075,commissioner ladies and gentlemen consumers it has just been said are one of the reasons for the existence of the european union,commissaris collegas consumenten het is net al gezegd zijn één van de bestaansredenen van de europese unie
193076,the committee on budgets will propose to the plenary that the budget for be restrictive,de begrotingscommissie zal de plenaire vergadering een restrictieve begroting voor voorstellen
193077,now however the selfsame commission albeit with a new commissioner is tabling a proposal for reforming the stability and growth pact the ambiguity of which will make it particularly welcome in the...,maar nu presenteert diezelfde commissie hoewel met een nieuwe commissaris een voorstel voor de hervorming van het stabiliteits en groeipact dat door zijn vaagheid vooral welkom zal zijn in de reke...
193078,mr president i refer to rule concerning translations,– mijnheer de voorzitter ik doe een beroep op artikel inzake vertalingen
